In [ ]:
# Führe diese Zelle einmal zum Start aus (Shift + Enter)
import piplite
# Wir laden unsere versteckte Hilfsdatei und Requests
await piplite.install(['requests', 'ipywidgets', 'micropip', 'pyodide-http'])
import utils

# Initialisiert das Netzwerk im Browser
await utils.setup_environment()

# UI für den API Key (damit er nicht im Code steht)
api_key_input = utils.widgets.Password(description="API Key:", placeholder="Hier Google Key einfügen")
display(api_key_input)

In [ ]:
# --- UNSERE LOGISTIK DATENBANK (Simuliert) ---

def check_container_status(container_id):
    """Prüft, wo sich ein Container befindet."""
    datenbank = {
        "CONT-HAM-01": "✅ Im Hamburger Hafen entladen (Zollfreigabe erteilt)",
        "CONT-MUC-99": "⚠️ LKW Panne auf A9 bei Ingolstadt (Verzögerung: 4h)",
        "CONT-CHN-88": "🚢 Auf See, ETA Rotterdam in 3 Tagen"
    }
    return datenbank.get(container_id, "❌ Container ID nicht im System gefunden.")

def berechne_transportkosten(gewicht_kg):
    """Berechnet Kosten basierend auf aktuellem Kerosinpreis."""
    preis = gewicht_kg * 2.50  # 2.50 EUR pro kg
    return f"{preis:.2f} EUR"

# Wir packen die Tools in eine Liste, damit der Agent sie findet
meine_tools = {
    "check_container_status": check_container_status,
    "berechne_transportkosten": berechne_transportkosten
}

print("Tools wurden definiert. Der Agent kann nun darauf zugreifen.")

In [ ]:
import json
from IPython.display import display, Markdown

# 1. Eingabefelder
frage_input = utils.widgets.Text(description="Frage:", placeholder="z.B. Wo ist CONT-MUC-99?", layout=utils.widgets.Layout(width='80%'))
start_btn = utils.widgets.Button(description="Agent starten", button_style='success')
output = utils.widgets.Output()

def run_agent(b):
    output.clear_output()
    api_key = api_key_input.value
    frage = frage_input.value
    
    with output:
        print(f"🤖 Agent analysiert: '{frage}' ...")
        
        # --- SCHRITT A: System Prompt definieren ---
        # Hier geben wir der KI ihre "Rolle" und erklären die Tools
        system_prompt = f"""
        Du bist ein Logistik-Disponent bei OSRAM.
        Du hast Zugriff auf folgende Funktionen:
        1. check_container_status(container_id)
        2. berechne_transportkosten(gewicht_kg)
        
        Szenario: Der Nutzer fragt: "{frage}"
        
        REGELN:
        - Wenn du Informationen aus den Funktionen brauchst, antworte NUR im JSON-Format: 
          {{"tool": "NAME_DER_FUNKTION", "arg": "ARGUMENT"}}
        - Wenn du die Antwort schon kennst oder es Smalltalk ist, antworte ganz normal.
        """
        
        # --- SCHRITT B: Erste Anfrage an Gemini ---
        antwort_1 = utils.call_gemini_api(api_key, system_prompt)
        
        # --- SCHRITT C: Prüfen, ob die KI ein Werkzeug nutzen will ---
        tool_befehl = utils.extract_json(antwort_1)
        
        if tool_befehl:
            tool_name = tool_befehl.get('tool')
            argument = tool_befehl.get('arg')
            print(f"🔧 KI entscheidet: Ich muss '{tool_name}' mit Wert '{argument}' nutzen.")
            
            # Werkzeug ausführen
            if tool_name in meine_tools:
                # Hier rufen wir unsere Python-Funktion aus Zelle 2 auf
                if tool_name == "berechne_transportkosten":
                    ergebnis = meine_tools[tool_name](float(argument))
                else:
                    ergebnis = meine_tools[tool_name](argument)
                    
                print(f"📊 Datenbank meldet: {ergebnis}")
                
                # --- SCHRITT D: Ergebnis zurück an die KI für finale Antwort ---
                finaler_prompt = f"""
                Der Nutzer fragte: "{frage}".
                Du wolltest das Tool nutzen. Das Ergebnis war: "{ergebnis}".
                Antworte dem Nutzer jetzt freundlich und professionell mit dieser Info.
                """
                finale_antwort = utils.call_gemini_api(api_key, finaler_prompt)
                display(Markdown(f"**Antwort:** {finale_antwort}"))
                
            else:
                print(f"Fehler: KI wollte unbekanntes Tool '{tool_name}' nutzen.")
        else:
            # Keine Tools nötig (z.B. bei "Hallo" oder allgemeinem Wissen)
            display(Markdown(f"**Antwort:** {antwort_1}"))

start_btn.on_click(run_agent)
display(frage_input, start_btn, output)